In [ ]:
import itertools

from datasets import load_dataset
import string
from collections import Counter
import re
import random

from utils import *

import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
from model import GPTConfig, GPT
import numpy as np

%load_ext autoreload
%autoreload 2

## Get the top words from a portion of the webtext dataset, write to file and load from

In [ ]:
# ## initial work with data to get the top words

# # # Load the OpenWebText dataset
# # openwebtext_subset = load_dataset("openwebtext", split="train[:1%]")

# # Initialize a Counter to count word occurrences
# word_counter = Counter()

# # Tokenize and count words in the dataset
# for i in range(len(openwebtext_subset)):
#     text = openwebtext_subset[i]['text']
    
#     # clean the text for case, punctuation, and numbers
#     # text = text.lower().translate(str.maketrans("", "", string.punctuation))
    
#     # Remove everything that is not a word character or a space, and convert to lowercase
#     text = re.sub(r"[^\w\s]", "", text.lower())
    
#     # Tokenize the text (you may want to use a more advanced tokenizer)
#     tokens = text.split()
#     # print(tokens)
    
#     # Update the counter with the tokens
#     word_counter.update(tokens)

# # Specify the number of top words for each case
# top_word_counts = [100, 1000, 10000]

# # Write the top words to files using the function
# for count in top_word_counts:
#     top_words = word_counter.most_common(count)
#     filename = f"top_{count}_words.txt"
#     write_top_words_to_file(top_words, filename)

# print("Top words written to files.")

In [ ]:
## load from the files

file_path = "top_1000_words.txt"  # Replace with the actual file path

# Read the lines from the file
with open(file_path, "r") as file:
    lines = file.readlines()

# Process each line and create a list of word-count pairs
top_1000_words = read_top_words_from_file("top_1000_words.txt")
top_10000_words= read_top_words_from_file("top_10000_words.txt")

print(top_1000_words[:5])
print(top_10000_words[:5])

## Create the mapping between words

In [ ]:
### create a mapping between 100 and 1000 top words
random.seed(42)

mapping = create_mapping(top_1000_words, top_10000_words)

# print an example mapping
print(f'key: {top_1000_words[0]} | vals: {mapping[top_1000_words[0]]}')

## Load GPT-2 Model

In [ ]:
model = load_GPT2()

In [ ]:
## taking a look at the vocabulary of GPT-2
enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)
decode(torch.arange(0, 50000, step=1000, dtype=None, layout=torch.strided, device=None, requires_grad=False).tolist())

# Generate Encyrptions and examine how likely they are

In [ ]:
print(mapping['drug'], mapping['meeting'])


In [ ]:
probs, encrypts = better_analysis(model, start="that's the beauty of ", secret_message='drug meeting', mapping=mapping)
plot_analysis(len(encrypts) - 1, len(encrypts) - 2, probs, encrypts, "drug-meeting.png")

In [ ]:
probs, encrypts = better_analysis(model, start='what if ', secret_message="drug meeting", mapping=mapping)
plot_analysis(len(encrypts) - 1, len(encrypts) - 2, probs, encrypts, "drug-meeting-ext.png")